## Define an External Batch Attribute Group (via SDK)

In [ ]:
from snowplow_signals import Signals
from dotenv import load_dotenv
import os

load_dotenv()

sp_signals = Signals(
    api_url=os.environ["SP_API_URL"],
    api_key=os.environ["SP_API_KEY"],
    api_key_id=os.environ["SP_API_KEY_ID"],
    org_id=os.environ["SP_ORG_ID"],
)

### Define a new data source

This block creates a data source for a Snowflake table.

In [ ]:
from snowplow_signals import BatchSource

data_source = BatchSource(
    name="product_recommendations",
    database="snowplow_sales_aws_prod1_db",
    schema="signals_test_derived",
    table="product_recommendations",
    timestamp_field="updated_at",
)

### Create an attribute group with the table fields

Attribute groups define the attributes inside the data sources and the attribute key types they relate to.

In [ ]:
from snowplow_signals import ExternalBatchAttributeGroup, user_id, Field

attribute_group = ExternalBatchAttributeGroup(
    name="product_recommendations",
    version=1,
    attribute_key=user_id,
    fields=[
        Field(
            name="main_interest",
            type="string",
        ),
        Field(
            name="recommendation_score",
            type="double",
        ),
        Field(
            name="loyalty_segment",
            type="string",
        ),
    ],
    batch_source=data_source,
    owner="user@company.com",
)

### Applying the data source and attribute group to Signals

The following block pushes the data source and attribute group definition to the Signals API and makes it available for a background CRON job that incrementally materializes the data from the warehouse table to the online attribute store.

In [ ]:
published = sp_signals.publish([attribute_group])
print(f"{len(published)} objects published")

# Wrap the attribute group in a service

In [ ]:
from snowplow_signals import Service

service = Service(
    name="batch_service",
    attribute_groups=[attribute_group],
    owner="user@company.com",
)

# Publish the service

Publish the attribute group to start processing it.

In [ ]:
sp_signals.publish([service])

# Retrieve attributes for the service

In [ ]:
service.get_attributes(
    signals=sp_signals,
    attribute_key="user_id",
    identifier="user_001",
)